In [3]:
%pip install openai ipython-ngql llama_index==0.8.9 pyvis

  Obtaining dependency information for llama_index==0.8.9 from https://files.pythonhosted.org/packages/f0/75/700dccd31ba85e81787447c2b4c1f9c9d0fe4d2207e86e308cc3e627a17b/llama_index-0.8.9-py3-none-any.whl.metadata
  Using cached pyvis-0.3.2-py3-none-any.whl (756 kB)
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Obtaining dependency information for jsonpickle>=1.4.1 from https://files.pythonhosted.org/packages/d3/25/6e0a450430b7aa194b0f515f64820fc619314faa289458b7dfca4a026114/jsonpickle-3.0.2-py3-none-any.whl.metadata
  Using cached soupsieve-2.4.1-py3-none-any.whl (36 kB)
   ---------------------------------------- 0.0/702.4 kB ? eta -:--:--
   - -------------------------------------- 30.7/702.4 kB 1.3 MB/s eta 0:00:01
   ------ --------------------------------- 112.6/702.4 kB 1.6 MB/s eta 0:00:01
   ------------- -------------------------- 235.5/702.4 kB 2.0 MB/s eta 0:00:01
   ------------------------- -------------- 440.3/702.4 kB 2.5 MB/s eta 0:00:01
   ---------

In [1]:
import os

openai_key = os.getenv("OPENAI_API_KEY")
openai_base = os.getenv("OPENAI_BASE")
openai_deployment = os.getenv("AZURE_OPEN_AI_MODEL")


os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_BASE"] = "https://mahesh.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "e7d4b167f002468491b093a02b187fd1"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"

In [23]:
import os
import json
import openai
from llama_index.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index import LangchainEmbedding
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
)

from llama_index.storage.storage_context import StorageContext
from llama_index.graph_stores import NebulaGraphStore
from llama_index.llms import LangChainLLM

import logging
import sys

from IPython.display import Markdown, display

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# openai.api_type = "azure"
# openai.api_base = "https://chat-contract-openai.openai.azure.com/"
# openai.api_version = "2023-05-15"
# os.environ["OPENAI_API_KEY"] = "631dd0eaef8340dabea0ae8a519b76be"
# openai.api_key = os.getenv("OPENAI_API_KEY")

openai.api_type = "azure"
openai.api_base = "https://prod-open-ai-service.openai.azure.com/"
openai.api_version = "2023-05-15"
os.environ["OPENAI_API_KEY"] = "30c82b416ff4456faa26b1644a83ab4b"
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = AzureOpenAI(
    engine="PROD-GPT-16K-TURBO",
    temperature=0,
    openai_api_version=openai.api_version,
    model_kwargs={
        "api_key": openai.api_key,
        "api_base": openai.api_base,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
    },
)

# You need to deploy your own embedding model as well as your own chat completion model
embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment="PROD-text-embedding-ada-002",
        openai_api_key=openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=16,
)

service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embedding_llm,
)

## Preparing NebulaGraph

### Create a nebulaspace space

In [3]:
!curl -fsSL nebula-up.siwei.io/install.sh | bash

^C


In [13]:
%load_ext ngql
connection_string = f"--address localhost --port 9669 --user root --password nebula"
%ngql {connection_string}
%ngql CREATE SPACE IF NOT EXISTS legalgraph(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);

The ngql extension is already loaded. To reload it, use:
  %reload_ext ngql
Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
[ERROR]:
 'IPythonNGQL' object has no attribute '_decode_value'
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


""


In [14]:
%ngql USE legalgraph;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


""


In [15]:
%%ngql
CREATE TAG IF NOT EXISTS entity(name string);
CREATE EDGE IF NOT EXISTS relationship(relationship string);
CREATE TAG INDEX IF NOT EXISTS entity_index ON entity(name(256));

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)
Get connection to ('127.0.0.1', 9669)


""


In [16]:
os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ[
    "NEBULA_ADDRESS"
] = "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally

space_name = "legalgraph"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [20]:
graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

## KG Building with Llama_Index

In [21]:
DATA_PATH = "../data-legal"

from llama_index import SimpleDirectoryReader

docs = SimpleDirectoryReader(DATA_PATH).load_data()

In [24]:
kg_index = KnowledgeGraphIndex.from_documents(
    docs,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    include_embeddings=True,
)

### Storing graph

In [25]:
kg_index.storage_context.persist(persist_dir='./storage_graph')


### Restoring storage context from storage

In [26]:
from llama_index import load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir='./storage_graph', graph_store=graph_store)
kg_index = load_index_from_storage(
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=5,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
    verbose=True,
)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


## KG_Index as query engine

In [27]:
kg_index_query_engine = kg_index.as_query_engine(
    retriever_mode="keyword",
    verbose=True,
    response_mode="tree_summarize",
)

In [28]:
response_graph_rag = kg_index_query_engine.query("What is the interest rate of the loan?")

display(Markdown(f"<b>{response_graph_rag}</b>"))

Extraced keywords: ['interest', 'rate', 'interest rate', 'loan']
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: e65e7b48-8368-449b-969e-eae416f84635: 34 
KL3 3346154.6Borrower shall and hereby promises to pay interest on the Pr...
> Querying with idx: e65e7b48-8368-449b-969e-eae416f84635: 34 
KL3 3346154.6Borrower shall and hereby promises to pay interest on the Pr...
> Querying with idx: e65e7b48-8368-449b-969e-eae416f84635: 34 
KL3 3346154.6Borrower shall and hereby promises to pay interest on the Pr...
> Querying with idx: e65e7b48-8368-449b-969e-eae416f84635: 34 
KL3 3346154.6Borrower shall and hereby promises to pay interest on the Pr...
INFO:llama_index.indices.knowledge_graph.retrievers:> Querying with idx: b60ade22-e1b6-43a7-b994-8c5a9ba077c0: 25 
KL3 3346154.6(C) if such Transfer would cause the transferee, together wi...
> Querying with idx: b60ade22-e1b6-43a7-b994-8c5a9ba077c0: 25 
KL3 3346154.6(C) if such Transfer would cause the transferee, togeth

<b>The interest rate of the loan is determined by the lender.</b>

## Graph RAG

In [29]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import KnowledgeGraphRAGRetriever

graph_rag_retriever = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    service_context=service_context,
    llm=llm,
    verbose=True,
)

query_engine = RetrieverQueryEngine.from_args(
    graph_rag_retriever, service_context=service_context
)

In [33]:
response = query_engine.query(
    "What is the name of the borrower?",
)
display(Markdown(f"<b>{response}</b>"))

Entities processed: ['name', 'borrower']
Entities processed: ['names', 'borrowers', 'name', 'borrower']
Graph RAG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...` extracted based on key entities as subject:
name, <-[enter]-, Lender, -[shall examine]->, books
name, <-[enter]-, Lender, -[shall examine]->, accounts
name, <-[enter]-, Lender, -[verify]->, performance
name, <-[enter]-, Lender, -[determination that]->, conditions to disbursement have been satisfied
name, <-[enter]-, Lender, -[shall return]->, Approved Letter of Credit
name, <-[enter]-, Lender, -[properly allocable to]->, the Property
name, <-[enter]-, Lender, -[has not received]->, written notice
name, <-[enter]-, Lender, -[subject to]->, terms and conditions
name, <-[enter]-, Lender, -[disburse funds held in]->, Funded Leasing Reserve
name, <-[enter]-, Lender, -[acquisition of]->, Property
name, <-[e

<b>The name of the borrower is not provided in the given context information.</b>

## Vector Store + Graph RAG

In [34]:
storage_context = StorageContext.from_defaults(graph_store=graph_store)

kg_index = KnowledgeGraphIndex.from_documents(
    docs,
    storage_context=storage_context,
    service_context=service_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

In [36]:
kg_index.storage_context.persist(persist_dir='./storage_graph')


In [32]:
from llama_index import VectorStoreIndex

service_context = ServiceContext.from_defaults(
    llm=llm,
)

vector_index = VectorStoreIndex.from_documents(docs, service_context=service_context)

RetryError: RetryError[<Future at 0x1714db2f2e0 state=finished raised InvalidRequestError>]

In [ ]:
# import QueryBundle
from llama_index import QueryBundle

# import NodeWithScore
from llama_index.schema import NodeWithScore

# Retrievers
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever

from typing import List


class CustomRetriever(BaseRetriever):
    """Custom retriever that performs both Vector search and Knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [ ]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine

# create custom retriever
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
custom_retriever = CustomRetriever(vector_retriever, kg_retriever)

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="tree_summarize",
)

### Create Query Engine

In [ ]:
custom_query_engine = RetrieverQueryEngine(
    retriever=custom_retriever,
    response_synthesizer=response_synthesizer,
)

vector_query_engine = vector_index.as_query_engine()

kg_keyword_query_engine = kg_index.as_query_engine(
    # setting to false uses the raw triplets instead of adding the text from the corresponding nodes
    include_text=False,
    retriever_mode="keyword",
    response_mode="tree_summarize",
)

In [ ]:
response = kg_keyword_query_engine.query("What is the name of the borrower in the agreement?")
display(Markdown(f"<b>{response}</b>"))